In [ ]:
# Setup: install Qiskit (runs automatically in Colab, no-op in Binder)
!pip install -q qiskit qiskit-aer qiskit-ibm-runtime pylatexenc

In [ ]:
# Additional dependencies for this notebook
!pip install -q gem-suite

*使用時間の目安: Heron r2 プロセッサで約3分（注: これは目安です。実際の実行時間は異なる場合があります。）*

## 背景

このチュートリアルでは、IBM&reg; 量子プロセッサ上で西森相転移を実現する方法を示します。この実験は、もともと[*Realizing the Nishimori transition across the error threshold for constant-depth quantum circuits*](https://arxiv.org/abs/2309.02863)で説明されたものです。

西森相転移とは、ランダムボンドイジングモデルにおける短距離秩序相と長距離秩序相の間の転移を指します。量子コンピュータ上では、長距離秩序相はデバイス全体にわたって量子ビットがエンタングルされた状態として現れます。この高度にエンタングルされた状態は、*測定によるエンタングルメント生成*（GEM: Generation of Entanglement by Measurement）プロトコルを用いて準備されます。GEMプロトコルは、回路中間測定を利用することで、定数深さの回路のみでデバイス全体の量子ビットをエンタングルすることができます。このチュートリアルでは、[GEM Suite](https://github.com/qiskit-community/gem-suite)ソフトウェアパッケージのGEMプロトコル実装を使用します。

## 前提条件

このチュートリアルを始める前に、以下がインストールされていることを確認してください：

- Qiskit SDK v1.0 以降（[visualization](https://docs.quantum.ibm.com/api/qiskit/visualization)サポート付き）
- Qiskit Runtime v0.22 以降（`pip install qiskit-ibm-runtime`）
- GEM Suite（`pip install gem-suite`）

## セットアップ

In [2]:
import matplotlib.pyplot as plt

from collections import defaultdict

from qiskit_ibm_runtime import QiskitRuntimeService

from qiskit.transpiler import generate_preset_pass_manager

from gem_suite import PlaquetteLattice
from gem_suite.experiments import GemExperiment

## ステップ 1: 古典的な入力を量子問題にマッピングする
GEMプロトコルは、格子で記述される量子ビット接続性を持つ量子プロセッサ上で動作します。現在のIBM量子プロセッサは[ヘビーヘックス格子](https://www.ibm.com/quantum/blog/heavy-hex-lattice)を使用しています。プロセッサの量子ビットは、格子の単位胞のどこに位置するかに基づいて*プラケット*にグループ分けされます。1つの量子ビットが複数の単位胞に含まれる場合があるため、プラケットは互いに素ではありません。ヘビーヘックス格子では、1つのプラケットに12個の量子ビットが含まれます。プラケット自体も格子を形成しており、2つのプラケットが量子ビットを共有している場合に接続されます。ヘビーヘックス格子では、隣接するプラケットは3個の量子ビットを共有します。

GEM Suiteソフトウェアパッケージにおいて、GEMプロトコルを実装するための基本クラスは `PlaquetteLattice` です。これはプラケットの格子を表現するもので、ヘビーヘックス格子とは異なるものです。`PlaquetteLattice` は量子ビットの結合マップから初期化できます。現在、ヘビーヘックス結合マップのみがサポートされています。

以下のコードセルでは、IBM量子プロセッサの結合マップからプラケット格子を初期化します。プラケット格子は必ずしもハードウェア全体を網羅するわけではありません。たとえば、`ibm_torino` は合計133個の量子ビットを持っていますが、デバイスに収まる最大のプラケット格子は125個の量子ビットのみを使用し、合計18個のプラケットで構成されます。異なる量子ビット数を持つ他のIBM Quantum&reg; デバイスでも同様の傾向が観察されます。

In [ ]:
# QiskitRuntimeService.save_account(channel="ibm_quantum", token="<YOUR_API_KEYN>", overwrite=True, set_as_default=True)
service = QiskitRuntimeService()
backend = service.least_busy(
    operational=True, simulator=False, min_num_qubits=127
)
plaquette_lattice = PlaquetteLattice.from_coupling_map(backend.coupling_map)

print(f"Number of qubits in backend: {backend.num_qubits}")
print(
    f"Number of qubits in plaquette lattice: {len(list(plaquette_lattice.qubits()))}"
)
print(f"Number of plaquettes: {len(list(plaquette_lattice.plaquettes()))}")

Number of qubits in backend: 133
Number of qubits in plaquette lattice: 125
Number of plaquettes: 18


You can visualize the plaquette lattice by generating a diagram of its graph representation. In the diagram, the plaquettes are represented by labeled hexagons, and two plaquettes are connected by an edge if they share qubits.

In [7]:
plaquette_lattice.draw_plaquettes()

<Image src="../docs/images/tutorials/nishimori-phase-transition/extracted-outputs/625882a4-faeb-4d96-b441-c989f43c4dea-0.avif" alt="Output of the previous code cell" />

プラケット格子のグラフ表現の図を生成することで、プラケット格子を可視化できます。図中では、プラケットはラベル付きの六角形で表され、2つのプラケットが量子ビットを共有している場合に辺で結ばれます。

In [8]:
# Get a list of the plaquettes
plaquettes = list(plaquette_lattice.plaquettes())
# Display information about plaquette 0
plaquettes[0]

PyPlaquette(index=0, qubits=[0, 1, 2, 3, 4, 15, 16, 19, 20, 21, 22, 23], neighbors=[3, 1])

![Output of the previous code cell](../docs/images/tutorials/nishimori-phase-transition/extracted-outputs/625882a4-faeb-4d96-b441-c989f43c4dea-0.avif)

`plaquettes` メソッドを使用して、個々のプラケットに含まれる量子ビットなどの情報を取得できます。

In [9]:
plaquette_lattice.draw_qubits()

<Image src="../docs/images/tutorials/nishimori-phase-transition/extracted-outputs/a19d63ce-3572-4081-a008-c1332fbbe303-0.avif" alt="Output of the previous code cell" />

In addition to the qubit labels and the edges indicating which qubits are connected, the diagram contains three additional pieces of information that are relevant to the GEM protocol:
- Each qubit is either shaded (gray) or unshaded. The shaded qubits are "site" qubits that represent the sites of the Ising model, and the unshaded qubits are "bond" qubits used to mediate interactions between the site qubits.
- Each site qubit is labeled either (A) or (B), indicating one of two roles a site qubit can play in the GEM protocol (the roles are explained later).
- Each edge is colored using one of six colors, thus partitioning the edges into six groups. This partitioning determines how two-qubit gates can be parallelized, as well as different scheduling patterns that are likely to incur different amounts of error on a noisy quantum processor. Because edges in a group are disjoint, a layer of two-qubit gates can be applied on those edges simultaneously. In fact, it is possible to partition the six colors into three groups of two colors such that the union of each group of two colors is still disjoint. Therefore, only three layers of two-qubit gates are needed to activate every edge. There are 12 ways to so partition the six colors, and each such partition yields a different 3-layer gate schedule.

Now that you have created a plaquette lattice, the next step is to initialize a `GemExperiment` object, passing both the plaquette lattice and the backend that you intend to run the experiment on. The `GemExperiment` class manages the actual implementation of the GEM protocol, including generating circuits, submitting jobs, and analyzing the data. The following code cell initializes the experiment class while restricting the plaquette lattice to only two of the plaquettes (21 qubits), reducing the size of the experiment to ensure that the noise in the hardware doesn't overwhelm the signal.

In [16]:
gem_exp = GemExperiment(plaquette_lattice.filter([9, 12]), backend=backend)

# visualize the plaquette lattice after filtering
plaquette_lattice.filter([9, 12]).draw_qubits()

<Image src="../docs/images/tutorials/nishimori-phase-transition/extracted-outputs/02357c6e-5c83-4ac0-811d-22602d9f33d5-0.avif" alt="Output of the previous code cell" />

プラケット格子を構成する基盤となる量子ビットの図も作成できます。

In [12]:
circuits = gem_exp.circuits()
print(f"Total number of circuits: {len(circuits)}")

Total number of circuits: 252


![Output of the previous code cell](../docs/images/tutorials/nishimori-phase-transition/extracted-outputs/a19d63ce-3572-4081-a008-c1332fbbe303-0.avif)

量子ビットのラベルと接続を示す辺に加えて、図にはGEMプロトコルに関連する3つの追加情報が含まれています：
- 各量子ビットは塗りつぶし（グレー）または塗りつぶしなしのいずれかです。塗りつぶされた量子ビットはイジングモデルのサイトを表す「サイト」量子ビットであり、塗りつぶされていない量子ビットはサイト量子ビット間の相互作用を媒介する「ボンド」量子ビットです。
- 各サイト量子ビットには (A) または (B) のラベルが付けられており、GEMプロトコルにおいてサイト量子ビットが果たす2つの役割のいずれかを示しています（役割については後述します）。
- 各辺は6色のうちの1色で色分けされており、辺を6つのグループに分割しています。この分割は、2量子ビットゲートの並列化方法と、ノイズの多い量子プロセッサ上で異なる量のエラーを引き起こす可能性のある異なるスケジューリングパターンを決定します。各グループ内の辺は互いに素であるため、それらの辺に対して2量子ビットゲートの層を同時に適用できます。実際には、6色を2色ずつ3つのグループに分割し、各2色グループの和集合が依然として互いに素になるようにすることが可能です。したがって、すべての辺を活性化するために必要な2量子ビットゲートの層は3層のみです。6色をこのように分割する方法は12通りあり、それぞれの分割が異なる3層のゲートスケジュールを生成します。

プラケット格子を作成したので、次のステップは `GemExperiment` オブジェクトを初期化することです。プラケット格子と実験を実行するバックエンドの両方を渡します。`GemExperiment` クラスは、回路の生成、ジョブの送信、データの分析を含むGEMプロトコルの実際の実装を管理します。以下のコードセルでは、プラケット格子を2つのプラケット（21量子ビット）のみに制限して実験クラスを初期化し、ハードウェアのノイズが信号を圧倒しないように実験のサイズを縮小しています。

In [13]:
# Restrict experiment to the first scheduling pattern
gem_exp.set_experiment_options(schedule_idx=0)

# There are less circuits now
circuits = gem_exp.circuits()
print(f"Total number of circuits: {len(circuits)}")

# Print the RZZ angles swept over
print(f"RZZ angles:\n{gem_exp.parameters()}")

Total number of circuits: 21
RZZ angles:
[0.         0.07853982 0.15707963 0.23561945 0.31415927 0.39269908
 0.4712389  0.54977871 0.62831853 0.70685835 0.78539816 0.86393798
 0.9424778  1.02101761 1.09955743 1.17809725 1.25663706 1.33517688
 1.41371669 1.49225651 1.57079633]


![Output of the previous code cell](../docs/images/tutorials/nishimori-phase-transition/extracted-outputs/02357c6e-5c83-4ac0-811d-22602d9f33d5-0.avif)

GEMプロトコル回路は、以下の手順で構築されます：
1. すべての量子ビットにアダマールゲートを適用して、全 $|+\rangle$ 状態を準備します。
2. 接続されたすべての量子ビットペアの間に $R_{ZZ}$ ゲートを適用します。これは3層のゲートで実現できます。各 $R_{ZZ}$ ゲートはサイト量子ビットとボンド量子ビットに作用します。サイト量子ビットのラベルが (B) の場合、角度は $\frac{\pi}{2}$ に固定されます。サイト量子ビットのラベルが (A) の場合、角度は可変であり、異なる回路を生成します。デフォルトでは、角度の範囲は $0$ から $\frac{\pi}{2}$ までの両端を含む等間隔の21点に設定されています。
3. 各ボンド量子ビットをパウリ $X$ 基底で測定します。量子ビットはパウリ $Z$ 基底で測定されるため、測定前にアダマールゲートを適用することでこれを実現できます。

このチュートリアルの冒頭で引用した論文では、$R_{ZZ}$ 角度に異なる規約を使用しており、このチュートリアルで使用する規約とは2倍の係数が異なることに注意してください。

ステップ3では、ボンド量子ビットのみが測定されます。サイト量子ビットがどのような状態に残されるかを理解するために、ステップ2でサイト量子ビット (A) に適用される $R_{ZZ}$ 角度が $\frac{\pi}{2}$ に等しい場合を考えると有益です。この場合、サイト量子ビットはGHZ状態に類似した高度にエンタングルされた状態に置かれます。

$$
\lvert \text{GHZ} \rangle = \lvert 00 \cdots 00 \rangle + \lvert 11 \cdots 11 \rangle.
$$

測定結果のランダム性により、サイト量子ビットの実際の状態は、たとえば $\lvert 00110 \rangle + \lvert 11001 \rangle$ のような、長距離秩序を持つ別の状態になる場合があります。しかし、測定結果に基づくデコード操作を適用することで、GHZ状態を回復できます。$R_{ZZ}$ 角度を $\frac{\pi}{2}$ から小さくしていくと、臨界角度まで長距離秩序を回復できます。ノイズがない場合、臨界角度は約 $0.3 \pi$ です。この角度を下回ると、得られる状態は長距離エンタングルメントを示さなくなります。この長距離秩序の有無の間の転移が、西森相転移です。

上記の説明では、サイト量子ビットは測定されずに残され、デコード操作は量子ゲートを適用することで実行できます。このチュートリアルが従うGEM Suiteで実装された実験では、実際にはサイト量子ビットも測定され、デコード操作は古典的な後処理ステップで適用されます。

上記の説明では、デコード操作はサイト量子ビットに量子ゲートを適用して量子状態を回復することで実行できます。しかし、たとえば特性評価の目的で状態を直ちに測定することが目的の場合、サイト量子ビットはボンド量子ビットとともに測定され、デコード操作は古典的な後処理ステップで適用できます。これがGEM Suiteでの実験の実装方法であり、このチュートリアルはこれに従います。

ステップ2の $R_{ZZ}$ 角度（デフォルトでは21個の値をスイープ）に加えて、GEMプロトコル回路は $R_{ZZ}$ ゲートの3層を実装するために使用されるスケジューリングパターンにも依存します。前述のとおり、このようなスケジューリングパターンは12通りあります。したがって、実験の回路総数は $21 \times 12 = 252$ です。

実験の回路は、`GemExperiment` クラスの `circuits` メソッドを使用して生成できます。

In [14]:
# Get the circuit at index 5
circuit = circuits[5]
# Remove the final measurements to ease visualization
circuit.remove_final_measurements()
# Draw the circuit
circuit.draw("mpl", fold=-1, scale=0.5)

<Image src="../docs/images/tutorials/nishimori-phase-transition/extracted-outputs/fd57d483-c70b-4ad5-b309-15750ad38bac-0.avif" alt="Output of the previous code cell" />

## Step 2: Optimize problem for quantum hardware execution

Transpiling quantum circuits for execution on hardware typically involves a [number of stages](/docs/guides/transpiler-stages). Typically, the stages that incur the most computational overhead are choosing the qubit layout, routing the two-qubit gates to conform to the qubit connectivity of the hardware, and optimizing the circuit to minimize its gate count and depth. In the GEM protocol, the layout and routing stages are unnecessary because the hardware connectivity is already incorporated into the design of the protocol. The circuits already have a qubit layout, and the two-qubit gates are already mapped onto native connections. Furthermore, in order to preserve the structure of the circuit as the $R_{ZZ}$ angle is varied, only very basic circuit optimization should be performed.

The `GemExperiment` class transparently transpiles circuits when executing the experiment. The layout and routing stages are already overridden by default to do nothing, and circuit optimization is performed at a level that only optimizes single-qubit gates. However, you can override or pass additional options using the `set_transpile_options` method. For the sake of visualization, the following code cell manually transpiles the circuit displayed previously, and draws the transpiled circuit.

In [15]:
# Demonstrate setting transpile options
gem_exp.set_transpile_options(
    optimization_level=1  # This is the default optimization level
)
pass_manager = generate_preset_pass_manager(
    backend=backend,
    initial_layout=list(gem_exp.physical_qubits),
    **dict(gem_exp.transpile_options),
)
transpiled = pass_manager.run(circuit)
transpiled.draw("mpl", idle_wires=False, fold=-1, scale=0.5)

<Image src="../docs/images/tutorials/nishimori-phase-transition/extracted-outputs/e9b99d48-8d33-46b5-bff5-480ab1c1c1f2-0.avif" alt="Output of the previous code cell" />

このチュートリアルでは、単一のスケジューリングパターンのみを考慮すれば十分です。以下のコードセルでは、実験を最初のスケジューリングパターンに制限します。その結果、実験にはスイープされる各 $R_{ZZ}$ 角度に1つずつ、合計21個の回路のみが含まれます。

In [10]:
exp_data = gem_exp.run(shots=10_000)

To wait for the results, call the `block_for_results` method of the `ExperimentData` object. This call will cause the interpreter to hang until the jobs are finished.

In [11]:
exp_data.block_for_results()

ExperimentData(GemExperiment, d0d5880a-34c1-4aab-a7b6-c4f58516bc03, job_ids=['cwg12ptmptp00082khhg'], metadata=<5 items>, figure_names=['two_point_correlation.svg', 'normalized_variance.svg', 'plaquette_ops.svg', 'bond_ops.svg'])

以下のコードセルでは、インデックス5の回路の図を描画します。図のサイズを縮小するため、回路末尾の測定ゲートは除去されています。

In [ ]:
def magnetization_distribution(
    counts_dict: dict[str, int],
) -> dict[str, float]:
    """Compute magnetization distribution from counts dictionary."""
    # Construct dictionary from magnetization to count
    mag_dist = defaultdict(float)
    for bitstring, count in counts_dict.items():
        mag = bitstring.count("0") - bitstring.count("1")
        mag_dist[mag] += count
    # Normalize
    shots = sum(counts_dict.values())
    for mag in mag_dist:
        mag_dist[mag] /= shots
    return mag_dist


# Get counts dictionaries with and without decoding
data = exp_data.data()
# Get the last data point, which is at the angle for the GHZ state
raw_counts = data[-1]["counts"]
# Without decoding
site_indices = [
    i for i, q in enumerate(gem_exp.plaquettes.qubits()) if q.role == "Site"
]
site_raw_counts = defaultdict(int)
for key, val in raw_counts.items():
    site_str = "".join(key[-1 - i] for i in site_indices)
    site_raw_counts[site_str] += val
# With decoding
_, site_decoded_counts = gem_exp.plaquettes.decode_outcomes(
    raw_counts, return_counts=True
)

# Compute magnetization distribution
raw_magnetization = magnetization_distribution(site_raw_counts)
decoded_magnetization = magnetization_distribution(site_decoded_counts)

# Plot
plt.bar(*zip(*raw_magnetization.items()), label="raw")
plt.bar(*zip(*decoded_magnetization.items()), label="decoded", width=0.3)
plt.legend()
plt.xlabel("Magnetization")
plt.ylabel("Frequency")
plt.title("Magnetization distribution with and without decoding")

Text(0.5, 1.0, 'Magnetization distribution with and without decoding')

<Image src="../docs/images/tutorials/nishimori-phase-transition/extracted-outputs/8ead3582-16df-4616-836c-bdce867ad6b8-1.avif" alt="Output of the previous code cell" />

![Output of the previous code cell](../docs/images/tutorials/nishimori-phase-transition/extracted-outputs/fd57d483-c70b-4ad5-b309-15750ad38bac-0.avif)

## ステップ 2: 量子ハードウェア実行のための問題の最適化
量子回路をハードウェア上で実行するためのトランスパイルには、通常[複数のステージ](/guides/transpiler-stages)が含まれます。一般的に、最も計算コストがかかるステージは、量子ビットレイアウトの選択、ハードウェアの量子ビット接続性に合わせた2量子ビットゲートのルーティング、およびゲート数と深さを最小化するための回路最適化です。GEMプロトコルでは、ハードウェアの接続性がすでにプロトコルの設計に組み込まれているため、レイアウトとルーティングのステージは不要です。回路にはすでに量子ビットレイアウトがあり、2量子ビットゲートはすでにネイティブ接続にマッピングされています。さらに、$R_{ZZ}$ 角度を変化させる際に回路の構造を保持するため、ごく基本的な回路最適化のみを実行する必要があります。

`GemExperiment` クラスは、実験の実行時に回路を透過的にトランスパイルします。レイアウトとルーティングのステージはデフォルトで何もしないようにオーバーライドされており、回路最適化は単一量子ビットゲートのみを最適化するレベルで実行されます。ただし、`set_transpile_options` メソッドを使用して、オーバーライドしたり追加のオプションを渡したりすることができます。可視化のため、以下のコードセルでは前に表示した回路を手動でトランスパイルし、トランスパイル後の回路を描画します。

In [13]:
exp_data.figure("two_point_correlation")

<Image src="../docs/images/tutorials/nishimori-phase-transition/extracted-outputs/4ecb25c8-e572-49af-a879-9943039db131-0.avif" alt="Output of the previous code cell" />

![Output of the previous code cell](../docs/images/tutorials/nishimori-phase-transition/extracted-outputs/e9b99d48-8d33-46b5-bff5-480ab1c1c1f2-0.avif)

## ステップ 3: Qiskitプリミティブを使用した実行
GEMプロトコル回路をハードウェア上で実行するには、`GemExperiment` オブジェクトの `run` メソッドを呼び出します。各回路からサンプリングするショット数を指定できます。`run` メソッドは [ExperimentData](https://qiskit-community.github.io/qiskit-experiments/stubs/qiskit_experiments.framework.ExperimentData.html) オブジェクトを返しますので、これを変数に保存してください。`run` メソッドはジョブの送信のみを行い、完了を待たないため、ノンブロッキング呼び出しであることに注意してください。

In [14]:
exp_data.figure("normalized_variance")

<Image src="../docs/images/tutorials/nishimori-phase-transition/extracted-outputs/2b351d68-3924-445a-94ef-047b16214e8a-0.avif" alt="Output of the previous code cell" />

結果を待つには、`ExperimentData` オブジェクトの `block_for_results` メソッドを呼び出します。この呼び出しにより、ジョブが完了するまでインタープリタはブロックされます。

In [15]:
gem_exp = GemExperiment(
    plaquette_lattice.filter(range(3, 9)), backend=backend
)
gem_exp.set_experiment_options(schedule_idx=0)
exp_data = gem_exp.run(shots=10_000)
exp_data.block_for_results()
exp_data.figure("normalized_variance")

<Image src="../docs/images/tutorials/nishimori-phase-transition/extracted-outputs/08581c09-a6a5-4a56-9fc4-abf22b063c6a-0.avif" alt="Output of the previous code cell" />

In [16]:
gem_exp = GemExperiment(plaquette_lattice, backend=backend)
gem_exp.set_experiment_options(schedule_idx=0)
exp_data = gem_exp.run(shots=10_000)
exp_data.block_for_results()
exp_data.figure("normalized_variance")

<Image src="../docs/images/tutorials/nishimori-phase-transition/extracted-outputs/37e9a4cd-6efb-4ade-ad09-8139db9d58e9-0.avif" alt="Output of the previous code cell" />

## ステップ 4: 後処理と所望の古典形式での結果の返却
$R_{ZZ}$ 角度が $\frac{\pi}{2}$ の場合、ノイズがなければデコードされた状態はGHZ状態になります。GHZ状態の長距離秩序は、測定されたビット列の磁化をプロットすることで可視化できます。磁化 $M$ は、単一量子ビットのパウリ $Z$ 演算子の和として定義されます。
$$
M = \sum_{j=1}^N Z_j,
$$
ここで $N$ はサイト量子ビットの数です。ビット列に対するその値は、0の数と1の数の差に等しくなります。GHZ状態を測定すると、全ゼロ状態または全1状態が等確率で得られるため、磁化は半分の確率で $+N$、もう半分の確率で $-N$ となります。ノイズによるエラーが存在する場合、他の値も現れますが、ノイズが大きすぎなければ、分布は依然として $+N$ と $-N$ の近くにピークを持ちます。

デコード前の生のビット列については、ノイズがない場合、磁化の分布は一様ランダムなビット列の分布と等価になります。

以下のコードセルでは、$R_{ZZ}$ 角度が $\frac{\pi}{2}$ の場合の生のビット列とデコードされたビット列の磁化をプロットします。